# **Email Spam Detection**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("https://github.com/9392473947/Datasets/raw/main/lingSpam.csv")

In [ ]:
data

,Unnamed: 0,Body,Label
0,0,Subject: great part-time or summer job !\n \n ...,1
1,1,Subject: auto insurance rates too high ?\n \n ...,1
2,2,Subject: do want the best and economical hunti...,1
3,3,Subject: email 57 million people for $ 99\n \n...,1
4,4,Subject: do n't miss these !\n \n attention ! ...,1
...,...,...,...
2600,2600,Subject: computationally - intensive methods i...,0
2601,2601,Subject: books : a survey of american linguist...,0
2602,2602,Subject: wecol ' 98 - - western conference on ...,0
2603,2603,Subject: euralex ' 98 - revised programme\n \n...,0


In [ ]:
data.shape

(2605, 3)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2605 entries, 0 to 2604
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2605 non-null   int64 
 1   Body        2605 non-null   object
 2   Label       2605 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 61.2+ KB


**DataPreprocessing**

In [ ]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

In [ ]:
def process_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [stemmer.stem(token) for token in tokens if token.isalpha() and token not in stop_words]
  return ' '.join(tokens)
processed_data = [process_text(doc) for doc in data.Body]

In [ ]:
data['preprocessed_text'] = data['Body'].apply(process_text)

# **Model Training**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(data['preprocessed_text'],data['Label'],test_size=0.2,random_state=42)

In [ ]:
len(X_train),len(X_test),len(y_train),len(y_test)

(2084, 521, 2084, 521)

In [ ]:
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

**Multinomial Naive Bayes**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

In [ ]:
classifier.fit(X_train_vectorized,y_train)

MultinomialNB()

In [ ]:
y_pred = classifier.predict(X_test_vectorized)

 **Evaluating the classifier on a test set**

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{classification_rep}')

Accuracy: 0.9942418426103646
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       436
           1       1.00      0.96      0.98        85

    accuracy                           0.99       521
   macro avg       1.00      0.98      0.99       521
weighted avg       0.99      0.99      0.99       521



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
def predict_sentiment(sample_review):
  sample_review = re.sub(pattern = '[^a-zA-Z]',repl=' ',string = sample_review)
  sample_review =sample_review.lower()
  sample_review_words = sample_review.split()
  sample_review_words = [word for word in sample_review_words if not word in set(stopwords.words('english'))]
  ps=PorterStemmer()
  final_review = [ps.stem(word) for word in sample_review_words]
  final_review = ' '.join(final_review)

  temp = vectorizer.transform([final_review]).toarray()
  return classifier.predict(temp)

In [ ]:
sample_mail = "“Subject: celtic studies"
if predict_sentiment(sample_mail):
  print('This is a SPAM Mail')
else:
  print('This is NOT SPAM Mail')

This is NOT SPAM Mail
